In [4]:
import os
import random
import shutil
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import albumentations as A
from tqdm.auto import tqdm
import cv2 # <--- Потрібен OpenCV

# ---------------------------------
# 1. КОНФІГУРАЦІЯ
# ---------------------------------
print("--- Етап 1: Конфігурація v2 ---")

FONT_DIR = './learning/fonts'
VALID_FONT_DIR = './learning/fonts'

# НОВА директорія для Класифікатора
DATA_DIR_V2 = './learning/data_classifier_v2'
TRAIN_DIR_V2 = os.path.join(DATA_DIR_V2, 'train')
TEST_DIR_V2 = os.path.join(DATA_DIR_V2, 'test')

FONT_SIZES = [40, 50, 60, 70, 80]
MAX_DIGITS_IN_NUMBER = 5
NUM_TRAIN_SCENES = 50000 # ~150к+ цифр
NUM_TEST_SCENES = 5000  # ~15к тестових цифр

if os.path.exists(DATA_DIR_V2):
    shutil.rmtree(DATA_DIR_V2)

# Створюємо 10 класів (0-9). Сміття нам не потрібне, якщо ми довіряємо YOLO
for c in list(range(10)):
    os.makedirs(os.path.join(TRAIN_DIR_V2, str(c)), exist_ok=True)
    os.makedirs(os.path.join(TEST_DIR_V2, str(c)), exist_ok=True)

print(f"Структуру директорій створено в {DATA_DIR_V2}")

# ---------------------------------
# 2. ДОПОМІЖНІ ФУНКЦІЇ (як у YOLO)
# ---------------------------------

def get_font_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.ttf')]

train_fonts = get_font_files(FONT_DIR)
valid_fonts = get_font_files(VALID_FONT_DIR)

def create_number_block(number_str, font_path, font_size):
    try: font = ImageFont.truetype(font_path, font_size)
    except IOError: font = ImageFont.load_default()
    bboxes = []
    images = []
    current_x = 0
    for char in number_str:
        bbox = font.getbbox(char)
        char_width = bbox[2] - bbox[0]
        char_height = bbox[3] - bbox[1]
        char_img = Image.new('L', (char_width + 4, font_size + 10), 0)
        draw = ImageDraw.Draw(char_img)
        draw.text((2 - bbox[0], 5 - bbox[1]), char, font=font, fill=255)
        x_min, y_min, x_max, y_max = current_x + 2, 5, current_x + 2 + char_width, 5 + char_height
        bboxes.append([int(char), x_min, y_min, x_max, y_max])
        images.append(char_img)
        current_x += char_img.width + random.randint(-5, 2)
    total_width, block_height = current_x, font_size + 10
    if total_width <= 0: return None, None
    block_img = Image.new('L', (total_width, block_height), 0)
    current_x = 0
    for img in images:
        block_img.paste(img, (current_x, 0))
        current_x += img.width + random.randint(-5, 2)
    return np.array(block_img), bboxes

block_transform = A.Compose([
    A.Perspective(scale=(0.02, 0.08), pad_mode=0, p=0.7),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=(-0.2, 0.2), rotate_limit=10, p=0.8),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

def preprocess_for_classifier(image_crop_gray, target_size=(64, 64)):
    h, w = image_crop_gray.shape[:2]
    if h == 0 or w == 0: return None
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = int(w * scale), int(h * scale)
    if new_h <= 0 or new_w <= 0: return None
    resized_img = cv2.resize(image_crop_gray, (new_w, new_h), interpolation=cv2.INTER_AREA)
    delta_w, delta_h = target_size[1] - new_w, target_size[0] - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)
    final_canvas = cv2.copyMakeBorder(resized_img, top, bottom, left, right,
                                     cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return final_canvas

# ---------------------------------
# 3. ГОЛОВНА ФУНКЦІЯ ГЕНЕРАЦІЇ
# ---------------------------------
print("--- Етап 2: Запуск генерації ---")
global_counter = 0

def generate_and_save(num_scenes, fonts, save_dir):
    global global_counter
    for _ in tqdm(range(num_scenes), desc=f"Генерація {save_dir}"):
        number_len = random.randint(1, MAX_DIGITS_IN_NUMBER)
        number_str = "".join([str(random.randint(0, 9)) for _ in range(number_len)])
        font_path, font_size = random.choice(fonts), random.choice(FONT_SIZES)
        block_img, bboxes = create_number_block(number_str, font_path, font_size)
        if block_img is None: continue
        class_labels = [b[0] for b in bboxes]
        pascal_bboxes = [b[1:] for b in bboxes]
        try:
            transformed = block_transform(image=block_img, bboxes=pascal_bboxes, class_labels=class_labels)
            transformed_img, transformed_bboxes, transformed_labels = transformed['image'], transformed['bboxes'], transformed['class_labels']
        except Exception:
            continue
        for label, bbox in zip(transformed_labels, transformed_bboxes):
            x1, y1, x2, y2 = map(int, bbox)
            cropped_digit = transformed_img[max(0, y1):int(y2), max(0, x1):int(x2)]
            final_64x64_image = preprocess_for_classifier(cropped_digit)
            if final_64x64_image is not None:
                save_path = os.path.join(save_dir, str(int(label)), f"digit_{global_counter}.png")
                Image.fromarray(final_64x64_image).save(save_path)
                global_counter += 1

# Генеруємо Train
generate_and_save(NUM_TRAIN_SCENES, train_fonts, TRAIN_DIR_V2)
# Генеруємо Test
generate_and_save(NUM_TEST_SCENES, valid_fonts, TEST_DIR_V2)

print(f"--- ✅ Генерацію завершено! Створено {global_counter} зображень у {DATA_DIR_V2} ---")

/tmp/ipykernel_11525/3427494308.py:75: UserWarning: Argument(s) 'pad_mode' are not valid for transform Perspective
  A.Perspective(scale=(0.02, 0.08), pad_mode=0, p=0.7),


--- Етап 1: Конфігурація v2 ---
Структуру директорій створено в ./learning/data_classifier_v2
--- Етап 2: Запуск генерації ---


Генерація ./learning/data_classifier_v2/test: 100%|█████████████████████████████████| 5000/5000 [01:34<00:00, 52.86it/s]

--- ✅ Генерацію завершено! Створено 102757 зображень у ./learning/data_classifier_v2 ---
